Create R.A.G model

In [1]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import UnstructuredPDFLoader



In [2]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
embedder  = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

In [4]:
os.listdir('../vector_db/')

['vectors_RecursiveCharacterTextSplitter_2000_200']

In [ ]:
llm_model_name = 'gpt-4-turbo'# "gpt-4o"
llm_temperature = 0.1

split_type = 'vectors_RecursiveCharacterTextSplitter_2000_200'
llm = ChatOpenAI(
    model=llm_model_name, 
    max_tokens=4096,
    temperature=llm_temperature,
    api_key=os.environ["OPENAI_API_KEY"],
)

In [6]:
db = FAISS.load_local(folder_path=f'../vector_db/{split_type}',
    embeddings=embedder,
    allow_dangerous_deserialization=True)

In [7]:
# retriever similarity 
retriever_similarity_5 = db.as_retriever(
            search_type='similarity',
            search_kwargs={'k': 5}
        )

In [8]:
template = f""""
System: Ти си асистент кој знае се за правилата на факултетот.
User: Одговори ми на прашањето со помош на текстот :

-----------
Ова е текстот:
{{context}}

-----------
Ова е прашањето:
{{question}}

Assistant:

"""

prompt = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

In [9]:
chain = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=retriever_similarity_5,
            chain_type_kwargs={"prompt": prompt, "verbose": True},
        )

In [17]:
question = 'Кога ја добива студентот дипломата?'
context = retriever_similarity_5.invoke(question)
answer = chain.invoke(
    question
)["result"]
# print(question, answer, context)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
"
System: Ти си асистент кој знае се за правилата на факултетот.
User: Одговори ми на прашањето со помош на текстот :

-----------
Ова е текстот:
обврски според студиската програма, врз 
основа на евиденцијата што се води за него, 
Универзитетот му издава диплома. 
Со дипломата се потврдува дека 
студентот завршил студиска програма од 
прв или втор циклус студии и се стекнува со 
соодветен стручен или академски назив. 
Составен дел на дипломата е 
додатокот на дипломата (Diploma 
Supplement). 
Дипломата ја потпишуваат ректорот 
и деканот. 
Додатокот на дипломата го 
потпишуваат ректорот и деканот. 
Дипломата и додатокот на 
дипломата се оверуваат со сув печат на 
Универзитетот.

изработка на дипломската работа. 
Оценката се носи со мнозинство гласови 
на членовите на комисијата. 
На студентот кој добил оценка 5 за 
дипломската работа, може да му се издаде 
уверение за положени испи

In [18]:
print(answer)


Студентот ја добива дипломата откако ќе ги заврши сите обврски според студиската програма, врз основа на евиденцијата што се води за него. Универзитетот му издава диплома која потврдува дека студентот успешно завршил студиска програма од прв или втор циклус студии и се стекнува со соодветен стручен или академски назив.
